http://klaksona.net/

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
from IPython.display import display
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

In [3]:
pd.set_option('display.max_colwidth', 1000)

In [4]:
oll_url=['http://klaksona.net/index.php?do=avtorynok-pmr&model_ids=Array&page=']

In [5]:
j=1
def parsing_2(url):
    data = pd.DataFrame(columns=['id', 'title', 'url', 'date_parsing', 'brend',  'объём', 'тип_двигателя', 'год', 'prise']) 
    reqx = requests.get('http://klaksona.net/index.php?do=avtorynok-pmr&model_ids=Array&page=1')
    soup = bs(reqx.text, 'html.parser')
    element = soup.findAll('a', class_="ajax_link")
    max_page = int(element[-8].text)
    for j in tqdm(range(1,max_page)):        
        reqx = requests.get('http://klaksona.net/index.php?do=avtorynok-pmr&model_ids=Array&page=' + str(j))
        soup = bs(reqx.text, 'html.parser')
        element = soup.findAll('div', class_='border_type_0')
        datax = []
        for i in element:
            datax.append({'id'  : i.find('a')['href'].split('/')[-1].replace('.html',''),
                          'title' : i.find('h1').text.lstrip(' ').rstrip(' '),
                          'url' : i.find('a')['href'],
                          'brend' : i.find('h1').text.split(' ')[2],
                          'date_parsing' : pd.to_datetime('today').normalize(),
                          'объём'  : i.find('tr').text.split('\r\n')[3].replace(' ',''),
                          'тип_двигателя' : i.find('tr').text.split('\r\n')[7].replace(' ',''),
                          'год' : i.find('td').text.lstrip(' ').rstrip(' ').split(' ')[1].replace('г.',''),
                          'prise' : str(i.find('tr').text.lstrip(' ').rstrip(' ').split(' ')[3]) + str(i.find('tr').text.lstrip(' ').rstrip(' ').split(' ')[4])
                         })
        datax = pd.DataFrame(datax)
        data = pd.concat([data, datax] , ignore_index=True)
    return data

In [6]:
df = parsing_2(oll_url)

  0%|          | 0/922 [00:00<?, ?it/s]

In [7]:
def add_details(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')
    element = soup.findAll('div', class_='container')
    lists = element[0].text.split('\n')
    if lists.count('Дополнительная информация:'):
        dop_info = lists.pop(lists.index('Дополнительная информация:') + 1).replace(' ','')
    else:
        dop_info = np.NaN
    if lists.count('Класс автомобиля:'):
        class_auto = lists.pop(lists.index('Класс автомобиля:') + 1).replace(' ','')
    else:
        class_auto = np.NaN
    if lists.count('Тип кузова:'):
        body = lists.pop(lists.index('Тип кузова:') + 1).replace(' ','')
    else:
        body = np.NaN
    if lists.count('Состояние:'):
        state = lists.pop(lists.index('Состояние:') + 1).replace(' ','')
    else:
        state = np.NaN  
    if lists.count('Кол-во дверей:'):
        door = lists.pop(lists.index('Кол-во дверей:') + 1).replace(' ','')
    else:
        door = np.NaN
    if lists.count('Потребляемое топливо:'):
        fuel = lists.pop(lists.index('Потребляемое топливо:') + 1).replace(' ','')
    else:
        fuel = np.NaN
    if lists.count('Коробка передач:'):
        transmission = lists.pop(lists.index('Коробка передач:') + 1).replace(' ','')
    else:
        transmission = np.NaN
    if lists.count('Кондиционер:'):
        conditioner = lists.pop(lists.index('Кондиционер:') + 1).replace(' ','')
    else:
        conditioner = np.NaN
    if lists.count('Аудиосистема:'):
        audio = lists.pop(lists.index('Аудиосистема:') + 1).replace(' ','')
    else:
        audio = np.NaN
    if lists.count('Пробег'):
        mileage = lists.pop(lists.index('Пробег') + 1).replace(' ','')
    else:
        mileage = np.NaN
    if lists.count('Контактное лицо'):
        contact = lists.pop(lists.index('Контактное лицо') + 1).replace(' ','')
    else:
        contact = np.NaN
    if lists.count('Телефон'):
        fone = lists.pop(lists.index('Телефон') + 1).replace(' ','')
    else:
        fone = np.NaN
    if lists.count('Размещено:'):
        date_notice = lists.pop(lists.index('Размещено:') + 1).replace(' ','')
    else:
        date_notice = np.NaN
    
    return pd.Series([dop_info, class_auto, body, state, door, fuel, transmission, conditioner, audio, mileage, contact, fone, date_notice])

In [8]:
 pd.options.mode.copy_on_write = True 

In [9]:
tqdm.pandas() 
df[['dop_info', 'class_auto', 'body', 'state', 'door', 'fuel',
   'transmission', 'conditioner', 'audio', 'mileage', 'contact',
   'fone', 'date_notice']] = df['url'].progress_apply(add_details)

  0%|          | 0/11045 [00:00<?, ?it/s]

In [10]:
df

,id,title,url,date_parsing,brend,объём,тип_двигателя,год,prise,dop_info,...,door,fuel,transmission,conditioner,audio,mileage,contact,fone,date_notice,mail
0,auto1683045,Nissan Sunny,http://klaksona.net/avtorynok-pmr/auto1683045.html,2024-09-02,Nissan,1600,Бензин,1993,1200,"ГОД1993ЦЕНА1200$МОТОР1,6БЕНЗИН,ПОЛНЫИЭЛПАКЕТ,ЭЛЗЕРКАЛА,СИГНАЛИЗАЦИЯ,ХОДОВАЯВНОРМЕПОЭЛЕКТРИКИВСЕРАБОТАЕТ,САЛОНАККУРАТНЫЙ,РЕЗИНАВСЕСЕЗОНКАСВЕЖИЕТ,О,СТРАХОВКАБЕЗОГРАНИЧЕНИЯ,ДЕЛАЮДОВЕРЕННОСТЬИЛИПЕРЕОФОРМЛЕНИЕМОБ077745154МОЛВОТСАП068653723",...,4/5,бензин,5-тиступ.мех.,NaN,автомагнитолаCD/MP3(безUSB),NaN,Денис,077745154,сегодняв15:51,NaN
1,auto1683044,Nissan Sunny,http://klaksona.net/avtorynok-pmr/auto1683044.html,2024-09-02,Nissan,1600,Бензин-Газ,1994,1350,NaN,...,4/5,бензин,5-тиступ.мех.,безкондиционера,автомагнитолаCD/MP3(безUSB),3,07797299,77997299,сегодняв15:48,NaN
2,auto1683043,Комплектующие двигатели и их запчасти,http://klaksona.net/avtorynok-pmr/auto1683043.html,2024-09-02,,неуказано,неуказано,2002,договорная,"РАЗБОРMercedes-BenzS-Класс3202002Г,",...,NaN,NaN,NaN,NaN,NaN,NaN,Лидия,77859505,сегодняв15:44,NaN
3,auto1683042,Комплектующие двигатели и их запчасти,http://klaksona.net/avtorynok-pmr/auto1683042.html,2024-09-02,,неуказано,неуказано,2002,договорная,"РАЗБОРMercedes-BenzS-Класс3202002Г,",...,NaN,NaN,NaN,NaN,NaN,NaN,Лидия,77859505,сегодняв15:43,NaN
4,auto1683041,Toyota Prius V,http://klaksona.net/avtorynok-pmr/auto1683041.html,2024-09-02,Toyota,1800,Гибрид,2012,8950,"ОТЛИЧНОЕАВТО,БЕЗВЛОЖЕНИЙ,РАСТАМОЖЕНО",...,4/5,бензин,автомат,кондиционер,автомагнитолаCD/MP3(безUSB),169000,Лидия,77859505,сегодняв15:41,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11040,auto1418130,Nissan X-Trail,http://klaksona.net/avtorynok-pmr/auto1418130.html,2024-09-02,Nissan,2200,Дизель,2006,5200,Автомобильсвежепригнанрастаможенотличноесостояние077785365069624860069606864,...,4/5,дизель,6-тиступ.мех.,климатконтроль,автомагнитолаCD/MP3(сUSBиBluetooth),207000,Олег(Всеобъявленияэтогопользователя),77766577,12.08.202216:39,NaN
11041,auto1418110,Toyota Camry,http://klaksona.net/avtorynok-pmr/auto1418110.html,2024-09-02,Toyota,2.5,Бензин,2014,10000,"Camryпокупалвмарте2020года(VINcode:4T1BF1FK4EU338342),стехпорпроводилтолькоТ/О(тормозныеколодкиилампочкифар),подкапотзаглядывалкрайнередко,толькочтобыпосмотретьуровеньмасла)МашинаэксплаутироваласьвосновномвгородеврежимеECO,тоестьбережноиаккуратно.За2годаназедилгде-то10Ккм+,потомучтоездилтолькопотерриторииПМРипонекоторымрегионамMD.Возможенуместныйторг,такжеучитвайте,чтомашинанаZ-номерах,которыевданныемоментневыпускают.Причинапродажи:покупкаавтоболеесвежегогодаБольшефото:https://disk.yandex.ru/a/XbjgA9uJPcPsIg",...,4/5,бензин,автомат,климатконтроль,автомагнитолаCD/MP3(сUSBиBluetooth),88000,Дмитрий(Всеобъявленияэтогопользователя),77593273,12.08.202215:38,NaN
11042,auto1418019,Audi A6,http://klaksona.net/avtorynok-pmr/auto1418019.html,2024-09-02,Audi,2500,Турбодизель,2000,3000,"ПродаетсяАУДИА6,2.5TDI2000,АКПП.Нейтральныеномера.АвтомобильнаходитсявБендерах.",...,4/5,дизель,автомат,климатконтроль,автомагнитолаCD/MP3(безUSB),345000,Вера(Всеобъявленияэтогопользователя),37377977765,12.08.202211:04,NaN
11043,auto1417999,Комплектующие двигатели и их запчасти,http://klaksona.net/avtorynok-pmr/auto1417999.html,2024-09-02,,неуказано,неуказано,1998,договорная,продаётсядвигательфордгалаксипозапчастям2.0бензин1998год,...,NaN,NaN,NaN,NaN,NaN,NaN,СергейИ.(Всеобъявленияэтогопользователя),077526883,12.08.202210:34,NaN


In [13]:
df.to_csv('csv/klakson'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False) 

In [14]:
preprocessing(df)

таблица имеет следующий вид:


,id,title,url,date_parsing,brend,объём,тип_двигателя,год,prise,dop_info,...,door,fuel,transmission,conditioner,audio,mileage,contact,fone,date_notice,mail
0,auto1683045,Nissan Sunny,http://klaksona.net/avtorynok-pmr/auto1683045.html,2024-09-02,Nissan,1600,Бензин,1993,1200,"ГОД1993ЦЕНА1200$МОТОР1,6БЕНЗИН,ПОЛНЫИЭЛПАКЕТ,ЭЛЗЕРКАЛА,СИГНАЛИЗАЦИЯ,ХОДОВАЯВНОРМЕПОЭЛЕКТРИКИВСЕРАБОТАЕТ,САЛОНАККУРАТНЫЙ,РЕЗИНАВСЕСЕЗОНКАСВЕЖИЕТ,О,СТРАХОВКАБЕЗОГРАНИЧЕНИЯ,ДЕЛАЮДОВЕРЕННОСТЬИЛИПЕРЕОФОРМЛЕНИЕМОБ077745154МОЛВОТСАП068653723",...,4/5,бензин,5-тиступ.мех.,NaN,автомагнитолаCD/MP3(безUSB),NaN,Денис,077745154,сегодняв15:51,NaN
1,auto1683044,Nissan Sunny,http://klaksona.net/avtorynok-pmr/auto1683044.html,2024-09-02,Nissan,1600,Бензин-Газ,1994,1350,NaN,...,4/5,бензин,5-тиступ.мех.,безкондиционера,автомагнитолаCD/MP3(безUSB),3,07797299,77997299,сегодняв15:48,NaN
2,auto1683043,Комплектующие двигатели и их запчасти,http://klaksona.net/avtorynok-pmr/auto1683043.html,2024-09-02,,неуказано,неуказано,2002,договорная,"РАЗБОРMercedes-BenzS-Класс3202002Г,",...,NaN,NaN,NaN,NaN,NaN,NaN,Лидия,77859505,сегодняв15:44,NaN
3,auto1683042,Комплектующие двигатели и их запчасти,http://klaksona.net/avtorynok-pmr/auto1683042.html,2024-09-02,,неуказано,неуказано,2002,договорная,"РАЗБОРMercedes-BenzS-Класс3202002Г,",...,NaN,NaN,NaN,NaN,NaN,NaN,Лидия,77859505,сегодняв15:43,NaN
4,auto1683041,Toyota Prius V,http://klaksona.net/avtorynok-pmr/auto1683041.html,2024-09-02,Toyota,1800,Гибрид,2012,8950,"ОТЛИЧНОЕАВТО,БЕЗВЛОЖЕНИЙ,РАСТАМОЖЕНО",...,4/5,бензин,автомат,кондиционер,автомагнитолаCD/MP3(безUSB),169000,Лидия,77859505,сегодняв15:41,NaN


****************************************************************************************************
таблица имеет 11045 сторок и 23 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:5
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


id                   0
title                0
url                  0
date_parsing         0
brend                0
объём                0
тип_двигателя        0
год                  0
prise                0
dop_info           105
class_auto        2784
body              3168
state             2959
door              3754
fuel              3316
transmission      3221
conditioner       4287
audio             5123
mileage           4052
contact              9
fone                 2
date_notice          2
mail             11045
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


id                 0.0
title              0.0
url                0.0
date_parsing       0.0
brend              0.0
объём              0.0
тип_двигателя      0.0
год                0.0
prise              0.0
dop_info           1.0
class_auto        25.2
body              28.7
state             26.8
door              34.0
fuel              30.0
transmission      29.2
conditioner       38.8
audio             46.4
mileage           36.7
contact            0.1
fone               0.0
date_notice        0.0
mail             100.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


id                       object
title                    object
url                      object
date_parsing     datetime64[ns]
brend                    object
объём                    object
тип_двигателя            object
год                      object
prise                    object
dop_info                 object
class_auto               object
body                     object
state                    object
door                     object
fuel                     object
transmission             object
conditioner              object
audio                    object
mileage                  object
contact                  object
fone                     object
date_notice              object
mail                    float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11045 entries, 0 to 11044
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             11045 non-null  object        
 1   title          11045 non-null  object        
 2   url            11045 non-null  object        
 3   date_parsing   11045 non-null  datetime64[ns]
 4   brend          11045 non-null  object        
 5   объём          11045 non-null  object        
 6   тип_двигателя  11045 non-null  object        
 7   год            11045 non-null  object        
 8   prise          11045 non-null  object        
 9   dop_info       10940 non-null  obj

None

In [16]:
df.groupby('id')['title'].count().sort_values(ascending=False)

id
auto1644601    2
auto1683005    2
auto1543210    2
auto1450483    2
auto1682973    2
              ..
auto1568633    1
auto1568670    1
auto1568734    1
auto1568765    1
auto799009     1
Name: title, Length: 11040, dtype: int64

In [25]:
df[df['id']=='auto1543210']

,id,title,url,date_parsing,brend,объём,тип_двигателя,год,prise,dop_info,...,door,fuel,transmission,conditioner,audio,mileage,contact,fone,date_notice,mail
8205,auto1543210,ГАЗ 3110,http://klaksona.net/avtorynok-pmr/auto1543210.html,2024-09-02,ГАЗ,неуказано,неуказано,1995,договорная,ПродамфаркопРАФ2203,...,NaN,NaN,NaN,NaN,NaN,NaN,Анатолий(Всеобъявленияэтогопользователя),077800477,12.07.202311:56,NaN
8206,auto1543210,ГАЗ 3110,http://klaksona.net/avtorynok-pmr/auto1543210.html,2024-09-02,ГАЗ,неуказано,неуказано,1995,договорная,ПродамфаркопРАФ2203,...,NaN,NaN,NaN,NaN,NaN,NaN,Анатолий(Всеобъявленияэтогопользователя),077800477,12.07.202311:56,NaN


In [26]:
reqx = requests.get('http://klaksona.net/avtorynok-pmr/auto1418110.html')
soup = bs(reqx.text, 'html.parser')
element = soup.findAll('div', class_='container')
element[0].text.split('\n')

['',
 '',
 '',
 '',
 '',
 'E-mail получателя:',
 '',
 '',
 '',
 '',
 ' Ваше Имя:',
 '',
 '',
 '',
 '',
 'Ваш E-mail:',
 '',
 '',
 '',
 '',
 ' Тема письма:',
 '',
 '',
 '',
 '',
 'Текст:',
 '',
 '',
 'Здравствуйте, .',
 '',
 'Обратите внимание на это авто: __auto_link__',
 '',
 'С уважением, ',
 '',
 'Авторынок - http://klaksona.net/',
 '',
 '',
 '\xa0',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' Toyota Camry \xa0  2014г. \xa0  ',
 '',
 '                        10 000 $ (торг уместен) \xa0 ',
 '',
 '                         \xa0 ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '  ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                                        \xa0 Количество просмотров обьявления: 3113 ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Toyota Camry\xa02014 г.\xa010 000 $, торг уместен',
 '',
 '',
 '',
 'Дополнительная информация:',
 '                                    Camry покупал в марте 2020 года(VIN cod